In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from pyproj import Transformer

### 지하철 승하차수

In [3]:
df_subway = pd.read_csv("data/subway.csv", encoding="euc-kr")

columns_to_select = [1, 3, 5] + list(range(6, df_subway.shape[1]))
df_subway = df_subway.iloc[:, columns_to_select]

df_subway.head()

,날짜,역번호,구분,06시 이전,06시-07시,07시-08시,08시-09시,09시-10시,10시-11시,11시-12시,...,15시-16시,16시-17시,17시-18시,18시-19시,19시-20시,20시-21시,21시-22시,22시-23시,23시-24시,24시 이후
0,2024-01-01,150,승차,383,257,308,975,1528,1720,2243,...,2716,2882,2871,2685,2922,2031,2279,1729,868,43
1,2024-01-01,150,하차,249,867,834,1201,1744,1731,2188,...,2615,2501,2829,2095,1833,1465,1031,585,298,82
2,2024-01-01,151,승차,188,92,167,245,322,357,503,...,935,1003,978,1116,951,909,723,462,176,3
3,2024-01-01,151,하차,103,276,292,451,614,575,754,...,834,697,670,564,351,302,272,120,89,38
4,2024-01-01,152,승차,970,374,193,205,260,344,620,...,1383,1578,1435,1422,1287,1318,1106,775,274,8


In [4]:
passenger_cols = [c for c in df_subway.columns
                  if c not in ['날짜','역번호','구분']]

df_subway['total_passengers'] = df_subway[passenger_cols].sum(axis=1)
columns_to_select = [0, 1, 2, 23]
df_subway= df_subway.iloc[:, columns_to_select]

# df_subway.columns == ['날짜','역번호','구분','total_passengers']

# 1) '날짜' → datetime → 'month' 컬럼 생성
df_subway['날짜'] = pd.to_datetime(df_subway['날짜'])
df_subway['month'] = df_subway['날짜'].dt.to_period('M').astype(str)  # "2024-01" 등

# 2) month, 역번호, 구분별로 월합(total_passengers 합계) 집계
df_subway = (
    df_subway
    .groupby(['역번호','구분','month'], as_index=False)
    .agg(monthly_total=('total_passengers','sum'))
)

# 3) 'month' → datetime → 해당 월의 일수 계산
df_subway['days_in_month'] = pd.to_datetime(
    df_subway['month'], format='%Y-%m'
).dt.days_in_month

# 4) 일평균 승객수 계산
df_subway['avg_daily_passengers'] = (
    df_subway['monthly_total'] / df_subway['days_in_month']
)

# 5) 최종 컬럼만 남기기
df_subway = df_subway[['month','역번호','구분','avg_daily_passengers']]

df_subway.head(30)


,month,역번호,구분,avg_daily_passengers
0,2024-01,150,승차,51564.258065
1,2024-02,150,승차,52722.206897
2,2024-03,150,승차,54228.903226
3,2024-04,150,승차,55824.933333
4,2024-05,150,승차,55546.064516
5,2024-06,150,승차,53273.033333
6,2024-07,150,승차,53633.709677
7,2024-08,150,승차,51978.000000
8,2024-09,150,승차,50544.033333
9,2024-10,150,승차,55854.516129


In [5]:
df_subway_coordinate = pd.read_csv("data/subway_coordinate.csv", encoding="euc-kr")
columns_to_select = [2, 4, 5]
df_subway_coordinate = df_subway_coordinate.iloc[:, columns_to_select]
df_subway_coordinate.head()

,고유역번호(외부역코드),위도,경도
0,150,37.553150,126.972533
1,151,37.563590,126.975407
2,152,37.570203,126.983116
3,153,37.570429,126.992095
4,154,37.570971,127.001900


In [ ]:
df_subway_merged = pd.merge(df_subway, df_subway_coordinate, 
                            left_on='역번호',
                            right_on='고유역번호(외부역코드)',
                            how='inner')

columns_to_select = [0, 1, 2, 3, 5, 6]
df_subway_merged = df_subway_merged.iloc[:, columns_to_select]

df_subway_merged.to_csv("data/subway_preprocessed.csv", index=False, encoding="utf-8-sig")
# 구분 기준으로 승차와 하차로 나누기
df_split = df_subway_merged.pivot_table(values='avg_daily_passengers', index=['month', '역번호', '위도', '경도'], columns='구분', aggfunc='first').reset_index()
df_split.rename(columns={'승차': 'avg_daily_board', '하차': 'avg_daily_alight'}, inplace=True)
df_split.to_csv("data/subway_preprocessed.csv", index=False)
df_split.head()


구분,month,역번호,위도,경도,avg_daily_board,avg_daily_alight
0,2024-01,150,37.553150,126.972533,51564.258065,49127.064516
1,2024-01,151,37.563590,126.975407,24446.096774,24504.096774
2,2024-01,152,37.570203,126.983116,37551.903226,36422.258065
3,2024-01,153,37.570429,126.992095,24925.322581,23167.032258
4,2024-01,154,37.570971,127.001900,22874.774194,22367.096774


### 버스 승하차수

In [9]:
months = [f"2024{month:02d}" for month in range(1, 7)]  
df_list = []


for ym in months:
    filename = f"data/bus/bus_{ym}.csv"

    df_temp = pd.read_csv(filename, encoding="euc-kr", low_memory=False)  
    df_temp["month"] = ym 
    
    df_list.append(df_temp)
    
df_bus = pd.concat(df_list, ignore_index=True)


columns_to_select = [0, 3] + list(range(6, 54))
df_bus = df_bus.iloc[:, columns_to_select]


df_bus.head()

,사용년월,표준버스정류장ID,00시승차총승객수,00시하차총승객수,1시승차총승객수,1시하차총승객수,2시승차총승객수,2시하차총승객수,3시승차총승객수,3시하차총승객수,...,19시승차총승객수,19시하차총승객수,20시승차총승객수,20시하차총승객수,21시승차총승객수,21시하차총승객수,22시승차총승객수,22시하차총승객수,23시승차총승객수,23시하차총승객수
0,202401,100000001,63,99,0,0,0,0,0,0,...,340,394,371,261,400,228,327,155,240,125
1,202401,100000001,0,0,0,0,68,97,215,137,...,0,0,0,0,0,0,0,0,0,0
2,202401,100000001,8,8,187,195,82,91,0,0,...,0,0,0,0,0,0,0,0,0,0
3,202401,100000001,162,123,77,147,0,0,0,0,...,338,489,278,256,273,160,252,187,196,138
4,202401,100000002,0,0,0,0,0,0,0,0,...,94,24,69,17,46,31,43,24,25,2


In [10]:
board_cols   = [c for c in df_bus.columns if '승차' in c]
alight_cols  = [c for c in df_bus.columns if '하차' in c]


df_bus['daily_board']  = df_bus[board_cols].sum(axis=1)
df_bus['daily_alight'] = df_bus[alight_cols].sum(axis=1)

df_daily = df_bus[[
    '사용년월',    
    '표준버스정류장ID',
    'daily_board',
    'daily_alight'
]]

df_daily.head()

,사용년월,표준버스정류장ID,daily_board,daily_alight
0,202401,100000001,7434,6513
1,202401,100000001,385,291
2,202401,100000001,277,294
3,202401,100000001,6150,7130
4,202401,100000002,2872,4011


In [13]:
df_daily_sum = (
    df_daily
    .groupby(['사용년월','표준버스정류장ID'], as_index=False)
    .agg({
        'daily_board': 'sum',
        'daily_alight': 'sum'
    })
)

df_daily_sum['days_in_month'] = pd.to_datetime(
    df_daily_sum['사용년월'].astype(str),
    format='%Y%m'
).dt.days_in_month

# 2) 일별 평균 승·하차 계산
df_daily_sum['avg_daily_board']   = df_daily_sum['daily_board']   / df_daily_sum['days_in_month']
df_daily_sum['avg_daily_alight'] = df_daily_sum['daily_alight'] / df_daily_sum['days_in_month']

# 3) 최종 컬럼 정리: 필요하면 days_in_month 제거
df_bus = df_daily_sum[[
    '사용년월',
    '표준버스정류장ID',
    'avg_daily_board',
    'avg_daily_alight'
]]

df_bus.head()

,사용년월,표준버스정류장ID,avg_daily_board,avg_daily_alight
0,202401,100000001,459.548387,458.967742
1,202401,100000002,2171.322581,1959.967742
2,202401,100000003,3464.258065,2535.161290
3,202401,100000004,680.096774,798.387097
4,202401,100000005,1598.290323,2307.161290


In [14]:
df_bus_coordinate = pd.read_excel("data/bus_coordinate.xlsx")

columns_to_select = [0, 3, 4]
df_bus_coordinate = df_bus_coordinate.iloc[:, columns_to_select]


df_bus_coordinate.head()

,NODE_ID,X좌표,Y좌표
0,100000001,126.987752,37.569806
1,100000002,126.996521,37.579433
2,100000003,126.998251,37.582580
3,100000004,126.987613,37.568579
4,100000005,127.001744,37.586243


In [21]:
df_bus_merged = pd.merge(df_bus, df_bus_coordinate, 
                            left_on='표준버스정류장ID',
                            right_on='NODE_ID',
                            how='inner')

df_bus_merged.rename(
    columns={
        'X좌표': '경도',
        'Y좌표': '위도',
        '사용년월':'month',
        '표준버스정류장ID':'ID'
    },
    inplace=True  # 원본 df를 바로 업데이트
)
new_cols = ['month', 'ID', 'avg_daily_board', 'avg_daily_alight', '위도', '경도']

#columns_to_select = [0, 1, 2, 3, 5, 6]
#df_bus_merged = df_bus_merged.iloc[:, columns_to_select]

df_bus_merged = df_bus_merged[new_cols]
df_bus_merged.to_csv("data/bus_preprocessed.csv", index=False, encoding="utf-8-sig")
df_bus_merged.head()


,month,ID,avg_daily_board,avg_daily_alight,위도,경도
0,202401,100000001,459.548387,458.967742,37.569806,126.987752
1,202401,100000002,2171.322581,1959.967742,37.579433,126.996521
2,202401,100000003,3464.258065,2535.161290,37.582580,126.998251
3,202401,100000004,680.096774,798.387097,37.568579,126.987613
4,202401,100000005,1598.290323,2307.161290,37.586243,127.001744
